In [2]:
using PCT

LoadError: ArgumentError: Package PCT not found in current path.
- Run `import Pkg; Pkg.add("PCT")` to install the PCT package.

In [3]:
h, _ = @pct pullback((z::C) -> z' * z)

(Pullback(MapType(VecType(AbstractPCTType[C(), C()]), C(), Dict{Any, Any}()), Map(MapType(VecType(AbstractPCTType[C()]), C(), Dict{Any, Any}()), PCTVector(VecType(AbstractPCTType[C()]), Var{C}[Var{C}(C(), :z)]), Mul(C(), PCTVector(VecType(AbstractPCTType[C(), C()]), PCT.AbstractPCTNode[Conjugate(C(), Var{C}(C(), :z)), Var{C}(C(), :z)])))), TypeContext(Dict{Symbol, Vector{<:AbstractPCTType}}(:_ => AbstractPCTType[UndeterminedPCTType()])))

In [4]:
println(verbose(h))

Pullback((z::C->
    (*
        z::C',
        z::C
    )::C
::[C->C])::[C×C->C]


In [5]:
reduce_pullback(h)[1]

(z, _K) -> 
    ((z, _K) -> 
        (((z, _K) -> 
            ((z, _K) -> 
                _K)(z, _K'))(z, (z'⋅_K))+((z, _K) -> 
            _K)(z, (_K⋅z))))(z, _K)

In [6]:
p = eval_all(reduce_pullback(h)[1])
q = eval_all(call(p, first(ff(p)), constant(1)))

(2.0⋅z)

In [2]:
f, ctx = @pct begin
    @space H begin
        type=(I, I)->C
        symmetries=(((2,1), :conj),)
    end
    
    @space V begin
        type=(I,)->C
    end

    @space T begin
        type=(I,I,I,I) -> C
        symmetries=(((2,1,4,3), :conj), ((3,4,1,2), :id))
    end
                
    @space U begin
        type=(I,I)->C
    end
                    
    (A::H, J::T) -> _
end

(Map(MapType(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))))]), UndeterminedPCTType(), Dict{Any, Any}()), PCTVector(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))))]), Var{MapType}[Var{MapType}(MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), :A), Var{MapType}(MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id)))), :J)]), Var{UndeterminedPCTTyp

In [11]:
f

(A, J) -> 
    _

In [39]:
g = @pct f ctx pullback((x::V) -> sum((i, j), x(i)' * A(i, j) * x(j)))

(A, J) -> 
    𝒫((x) -> 
        ∑((i_0, i_1), (x(i_0)⋅A(i_1, i_0)⋅x(i_1)')))

In [40]:
println(verbose(g))

(A::[I×I->C], J::[I×I×I×I->C]->
    Pullback((x::[I->C]->
        ∑((i_0::I, i_1::I),
            (*
                x::[I->C](i_0::I)::C,
                A::[I×I->C](i_1::I, i_0::I)::C,
                x::[I->C](i_1::I)::C'
            )::C
        )::C
    ::[[I->C]->C])::[[I->C]×C->[I->C]]
::[[I×I->C]×[I×I×I×I->C]->[[I->C]×C->[I->C]]]


In [41]:
p = fc(eval_all(reduce_pullback(g)))

(x, _K) -> 
    (i_2) -> 
        ∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)⋅_K'))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'⋅_K))))

In [42]:
q = eval_all(call(p, first(ff(p)), constant(1)))

(i_2) -> 
    ∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))

In [43]:
simplify(q) |> first

  0.000019 seconds (7 allocations: 304 bytes)
1 sum_dist
∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)')))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))

  0.000281 seconds (293 allocations: 13.062 KiB)
2 contract_delta
(∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)')))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
directed!

  0.000041 seconds (215 allocations: 9.688 KiB)
3 contract_delta
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0), (x(i_0)⋅A(i_2, i_0))))
directed!

  0.000019 seconds (37 allocations: 1.625 KiB)
4 gcd
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0), (x(i_0)⋅A(i_2, i_0))))
((1+1)⋅∑((i_0), (x(i_0)⋅A(i_2, i_0))))

  0.000042 seconds (244 allocations: 11.312 KiB)
5 add_const
((1+1)⋅∑((i_0), (x(i_0)⋅A(i_2, i_0))))
(∑((i_0), (x

(i_2) -> 
    (∑((i_0), (x(i_0)⋅A(i_2, i_0)))⋅2)

In [44]:
g = @pct f ctx pullback((C::U) -> sum((i,j,p,q,r,s), C(p,i)' * C(q,i) * C(r,j)' * C(s,j) * J(p,q,r,s)))

(A, J) -> 
    𝒫((C) -> 
        ∑((i_0, i_1, i_2, i_3, i_4, i_5), (C(i_1, i_5)⋅C(i_3, i_4)⋅J(i_1, i_0, i_3, i_2)'⋅C(i_2, i_4)'⋅C(i_0, i_5)')))

In [45]:
p = fc(eval_all(reduce_pullback(g)))

(C, _K) -> 
    (i_6, i_7) -> 
        ∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_7, i_5, δ(i_6, i_2, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_1, i_0)'⋅_K)))+δ(i_7, i_5, δ(i_6, i_3, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_4, i_0)'⋅_K')))+δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)'⋅_K)))+δ(i_7, i_0, δ(i_6, i_4, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_3, i_5)'⋅_K')))))

In [46]:
q = eval_all(call(p, first(ff(p)), constant(1)))

(i_6, i_7) -> 
    ∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_7, i_5, δ(i_6, i_2, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_1, i_0)')))+δ(i_7, i_5, δ(i_6, i_3, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_4, i_0)')))+δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)')))+δ(i_7, i_0, δ(i_6, i_4, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_3, i_5)')))))

In [8]:
Profile.clear()
@profile p==p
pprof()

"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599


In [ ]:
(A::H) -> (x::V) -> x A x

In [7]:
f, ctx = @pct begin 

    @domain P begin
        base = I
        lower = -N
        upper = N-1
        periodic = true
    end

    @domain Q begin
        base = I
        lower = -N
        upper = N
        contractable = false
    end

    @space Mmn begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), )
    end

    @space Sym begin
        type = (I, ) -> C
        symmetries = (((1, ), :ineg), )
    end

    @space Gauge begin
        type = (I, I, I) -> C
    end

    @space Density begin
        type = (I, I) -> C
    end

    (S::Mmn, w::Sym) -> _
end

g = @pct f ctx pullback((U::Gauge) -> ((ρ::Density) -> sum((n::I, b::Q), ρ(n, b)' * ρ(n, b)))(
    (n::I, b::Q) -> sum((k::P, p, q), U(p, n, k)' * S(p,q,k,k+b) * U(q, n, k+b))))
eval_all(g)
g_1 = fc(eval_all(reduce_pullback(eval_all(g))))
g_2 = eval_all(call(g_1, first(ff(g_1)), constant(1)))

(i_0, i_1, i_2) -> 
    ∑((n, b), (∑((k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))⋅S(p, q, k, (b+k))'⋅U(p, n, k)))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (∑((k, p, q), (U(q, n, (b+k))'⋅S(p, q, k, (b+k))'⋅U(p, n, k)))⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))))))⋅2.0))

In [9]:
g_3 = simplify(g_2; settings=Dict(:clench_sum=>true)) |> first

  0.011581 seconds (93.74 k allocations: 3.738 MiB)
1 sum_out
∑((n, b), (∑((k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))⋅S(p, q, k, (b+k))'⋅U(p, n, k)))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (∑((k, p, q), (U(q, n, (b+k))'⋅S(p, q, k, (b+k))'⋅U(p, n, k)))⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))))))⋅2.0))
∑((b), (∑((n, k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))⋅S(p, q, k, (b+k))'⋅U(p, n, k)))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (∑((k, p, q), (U(q, n, (b+k))'⋅S(p, q, k, (b+k))'⋅U(p, n, k)))⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))))))⋅2.0))
yesdirected!

  0.154785 seconds (354.42 k allocations: 21.254 MiB, 90.37% compilation time)
2 sum_out
∑((b), (∑((n, k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))⋅S(p, q, k, (b+k))'⋅U(p, n, k)))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (∑((k, p, q), (U(q, n, (b+k))'⋅S(p, q, k, (b+k))'⋅U(p, n, k)))⋅U

(i_0, i_1, i_2) -> 
    (2.0⋅(∑((b, q), (∑((k, q), (∑((p), (S(p, q, k, (b+k))'⋅U(p, i_1, k)))⋅U(q, i_1, (b+k))'))⋅U(q, i_1, (i_2+b))⋅S(i_0, q, i_2, (i_2+b))))+∑((b, p), (∑((k, q), (∑((p), (U(p, i_1, k)'⋅S(p, q, k, (b+k))))⋅U(q, i_1, (b+k))))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'⋅U(p, i_1, ((-1.0⋅b)+i_2))))))

In [12]:
g_4 = simplify(g_3; settings=Dict(:symmetry=>true)) |> first

  0.040604 seconds (214.69 k allocations: 11.719 MiB, 84.82% compilation time)
1 sum_in
(2.0⋅(∑((b, q), (∑((k, q), (∑((p), (S(p, q, k, (b+k))'⋅U(p, i_1, k)))⋅U(q, i_1, (b+k))'))⋅U(q, i_1, (i_2+b))⋅S(i_0, q, i_2, (i_2+b))))+∑((b, p), (∑((k, q), (∑((p), (U(p, i_1, k)'⋅S(p, q, k, (b+k))))⋅U(q, i_1, (b+k))))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'⋅U(p, i_1, ((-1.0⋅b)+i_2))))))
(2.0⋅(∑((b, p), (∑((k, q), (∑((p), (U(p, i_1, k)'⋅S(p, q, k, (b+k))))⋅U(q, i_1, (b+k))))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'⋅U(p, i_1, ((-1.0⋅b)+i_2))))+∑((b, q, k, i_3), (∑((p), (S(p, i_3, k, (b+k))'⋅U(p, i_1, k)))⋅U(i_3, i_1, (b+k))'⋅U(q, i_1, (i_2+b))⋅S(i_0, q, i_2, (i_2+b))))))
yesdirected!

  0.008010 seconds (129.97 k allocations: 3.866 MiB)
2 sum_in
(2.0⋅(∑((b, p), (∑((k, q), (∑((p), (U(p, i_1, k)'⋅S(p, q, k, (b+k))))⋅U(q, i_1, (b+k))))⋅S(p, i_0, ((-1.0⋅b)+i_2), i_2)'⋅U(p, i_1, ((-1.0⋅b)+i_2))))+∑((b, q, k, i_3), (∑((p), (S(p, i_3, k, (b+k))'⋅U(p, i_1, k)))⋅U(i_3, i_1, (b+k))'⋅U(q, i_1, (i_2+b))⋅S(i_0, q, i_2, (i_2+b)))))

(i_0, i_1, i_2) -> 
    ∑((b, p, k, q, i_3), (4.0⋅U(i_3, i_1, (b+k))'⋅U(q, i_1, k)⋅U(p, i_1, (i_2+b))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (i_2+b))))

In [13]:
g_5 = simplify(g_4; settings=Dict(:clench_sum=>true)) |> first

  0.000038 seconds (186 allocations: 7.266 KiB)
1 sum_out
∑((b, p, k, q, i_3), (4.0⋅U(i_3, i_1, (b+k))'⋅U(q, i_1, k)⋅U(p, i_1, (i_2+b))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (i_2+b))))
∑((p, k, q, i_3), (∑((b), (U(i_3, i_1, (b+k))'⋅U(p, i_1, (i_2+b))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (i_2+b))))⋅4.0⋅U(q, i_1, k)))
yesdirected!

  0.000053 seconds (293 allocations: 11.453 KiB)
2 sum_out
∑((p, k, q, i_3), (∑((b), (U(i_3, i_1, (b+k))'⋅U(p, i_1, (i_2+b))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (i_2+b))))⋅4.0⋅U(q, i_1, k)))
∑((k, q, i_3), (∑((p, b), (U(i_3, i_1, (b+k))'⋅U(p, i_1, (i_2+b))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (i_2+b))))⋅4.0⋅U(q, i_1, k)))
yesdirected!

  0.000439 seconds (3.26 k allocations: 146.250 KiB)
3 sum_out
∑((k, q, i_3), (∑((p, b), (U(i_3, i_1, (b+k))'⋅U(p, i_1, (i_2+b))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (i_2+b))))⋅4.0⋅U(q, i_1, k)))
∑((q, i_3), (∑((k), (∑((p, b), (U(i_3, i_1, (b+k))'⋅U(p, i_1, (i_2+b))⋅S(i_3, q, (b+k), k)⋅S(i_0, p, i_2, (i_2+b))))⋅U(q, i_1, k)))⋅4.0))
yesdir

(i_0, i_1, i_2) -> 
    (∑((q, k), (∑((b), (∑((i_3), (U(i_3, i_1, (b+k))'⋅S(i_3, q, (b+k), k)))⋅∑((p), (U(p, i_1, (i_2+b))⋅S(i_0, p, i_2, (i_2+b))))))⋅U(q, i_1, k)))⋅4.0)